# Recurring Transfers Analysis (Daily / Weekly / Monthly)
This notebook loads a `transactions` file (CSV or Excel), computes daily/weekly/monthly recurrence for each `(AccountNumber, CorrespondentAccountNumber)` pair over the dataset timeframe, **filters pairs that have 100% recurrence**, and generates six visualizations:

1. Calendar-style heatmap (per pair)  
2. Timeline/line plot of transfer presence or counts  
3. Heatmap (pairs × days/weeks)  
4. Network graph of recurring pairs  
5. Bar chart of top recurring pairs  
6. Amount-based recurrence / stability check

**How to use**:  
- Place your `transactions.csv` or `transactions.xlsx` in the same directory as this notebook before running.  
- The notebook expects columns: `AccountNumber`, `CorrespondentAccountNumber`, `ValueDate` (case-insensitive). If your column names differ, adapt the `rename` mapping in the "Load & normalize" cell.

Files created by the notebook:
- `/mnt/data/recurrence_results.xlsx` — summary tables of recurring pairs.


In [ ]:
# Load libraries and data (tries CSV then Excel)
import os
import pandas as pd
from datetime import datetime

# Try common filenames
candidates = ['transactions.csv', 'transactions.xlsx', 'transactions.xls']
found = None
for fn in candidates:
    if os.path.exists(fn):
        found = fn
        break

if found is None:
    raise FileNotFoundError(
        "No transactions file found. Please upload 'transactions.csv' or 'transactions.xlsx' into the notebook working directory."
    )

print(f"Loading file: {found}")
if found.lower().endswith('.csv'):
    df = pd.read_csv(found, parse_dates=['ValueDate'] if 'ValueDate' in pd.read_csv(found, nrows=0).columns else None)
else:
    df = pd.read_excel(found, parse_dates=['ValueDate'] if 'ValueDate' in pd.read_excel(found, nrows=0).columns else None)

# normalize column names (case-insensitive)
df.columns = [c.strip() for c in df.columns]
col_map = {}
for c in df.columns:
    lc = c.lower()
    if lc in ('accountnumber','account','from_account','fromacct'):
        col_map[c] = 'AccountNumber'
    if lc in ('correspondentaccountnumber','corr_account','to_account','toacct','correspondent'):
        col_map[c] = 'CorrespondentAccountNumber'
    if lc in ('valuedate','value_date','value','date','ts','timestamp'):
        col_map[c] = 'ValueDate'
    if lc in ('amount','amt','transactionamount'):
        col_map[c] = 'Amount'

df = df.rename(columns=col_map)
required = ['AccountNumber','CorrespondentAccountNumber','ValueDate']
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}. Please rename your columns to include AccountNumber, CorrespondentAccountNumber, and ValueDate.")

# Ensure ValueDate is datetime
df['ValueDate'] = pd.to_datetime(df['ValueDate'])

# create bucket columns
df['date'] = df['ValueDate'].dt.date
df['week'] = df['ValueDate'].dt.to_period('W').apply(lambda r: r.start_time.date())
df['month'] = df['ValueDate'].dt.to_period('M').apply(lambda r: r.start_time.date())

print('Data loaded. Rows:', len(df))
df.head(3)


In [ ]:
# Compute total buckets and counts per pair
total_days = df['date'].nunique()
total_weeks = df['week'].nunique()
total_months = df['month'].nunique()

print('Total days:', total_days, 'Total weeks:', total_weeks, 'Total months:', total_months)

daily_counts = df.groupby(['AccountNumber','CorrespondentAccountNumber'])['date'].nunique().reset_index(name='days_with_transfer')
weekly_counts = df.groupby(['AccountNumber','CorrespondentAccountNumber'])['week'].nunique().reset_index(name='weeks_with_transfer')
monthly_counts = df.groupby(['AccountNumber','CorrespondentAccountNumber'])['month'].nunique().reset_index(name='months_with_transfer')

final_df = daily_counts.merge(weekly_counts, on=['AccountNumber','CorrespondentAccountNumber'], how='outer') \
    .merge(monthly_counts, on=['AccountNumber','CorrespondentAccountNumber'], how='outer')

final_df[['days_with_transfer','weeks_with_transfer','months_with_transfer']] = final_df[['days_with_transfer','weeks_with_transfer','months_with_transfer']].fillna(0).astype(int)

final_df['total_days'] = total_days
final_df['total_weeks'] = total_weeks
final_df['total_months'] = total_months

final_df['pct_daily'] = final_df['days_with_transfer'] / total_days * 100
final_df['pct_weekly'] = final_df['weeks_with_transfer'] / total_weeks * 100
final_df['pct_monthly'] = final_df['months_with_transfer'] / total_months * 100

# Save full summary
final_df_sorted = final_df.sort_values(['pct_weekly','pct_daily','pct_monthly'], ascending=False)
final_df_sorted.head(10)


In [ ]:
# Filter pairs with 100% recurrence in any of daily/weekly/monthly
daily_100 = final_df[(final_df['pct_daily'] == 100)].copy().reset_index(drop=True)
weekly_100 = final_df[(final_df['pct_weekly'] == 100)].copy().reset_index(drop=True)
monthly_100 = final_df[(final_df['pct_monthly'] == 100)].copy().reset_index(drop=True)

print('Pairs with 100% daily recurrence:', len(daily_100))
print('Pairs with 100% weekly recurrence:', len(weekly_100))
print('Pairs with 100% monthly recurrence:', len(monthly_100))

# Save results to Excel for easy download
output_path = '/mnt/data/recurrence_results.xlsx'
with pd.ExcelWriter(output_path) as xw:
    final_df_sorted.to_excel(xw, sheet_name='all_pairs', index=False)
    daily_100.to_excel(xw, sheet_name='daily_100', index=False)
    weekly_100.to_excel(xw, sheet_name='weekly_100', index=False)
    monthly_100.to_excel(xw, sheet_name='monthly_100', index=False)

print('Saved results to', output_path)


In [ ]:
# Visualizations
# We'll generate visualizations for pairs with 100% recurrence.
# To avoid excessive plots, cap the number visualized per recurrence type to N (default 20).
import matplotlib.pyplot as plt
import math
from textwrap import wrap
try:
    import networkx as nx
    nx_available = True
except Exception:
    nx_available = False

VISUAL_CAP = 20

def pairs_from_df(df_pairs, cap=VISUAL_CAP):
    pairs = list(zip(df_pairs['AccountNumber'].astype(str), df_pairs['CorrespondentAccountNumber'].astype(str)))
    return pairs[:cap]

daily_pairs = pairs_from_df(daily_100)
weekly_pairs = pairs_from_df(weekly_100)
monthly_pairs = pairs_from_df(monthly_100)

print('Will visualize up to', VISUAL_CAP, 'pairs per recurrence bucket (if available).')

In [ ]:
# 1) Calendar-style heatmap per pair (daily presence)
# For each pair, create a simple heatline plot showing presence across the date range.
def plot_calendar_heatline(pair, df, ax=None):
    a, b = pair
    mask = (df['AccountNumber'].astype(str) == str(a)) & (df['CorrespondentAccountNumber'].astype(str) == str(b))
    s = df[mask].copy()
    if s.empty:
        return None
    idx = pd.date_range(df['ValueDate'].min().date(), df['ValueDate'].max().date(), freq='D')
    presence = s.groupby(s['ValueDate'].dt.date).size().reindex(idx.date, fill_value=0)
    if ax is None:
        fig, ax = plt.subplots(figsize=(12,1.5))
    ax.plot(idx, (presence>0).astype(int), linewidth=1)
    ax.set_ylim(-0.1,1.1)
    ax.set_yticks([0,1])
    ax.set_yticklabels(['no','yes'])
    ax.set_title(f'Presence by day for pair {a} -> {b}')
    ax.set_xlabel('Date')
    return ax

# Example: create a multi-plot figure for first few daily pairs
n = len(daily_pairs)
if n>0:
    cols = 1
    rows = n
    fig = plt.figure(figsize=(12, 1.5*rows))
    for i, pair in enumerate(daily_pairs, start=1):
        ax = fig.add_subplot(rows, cols, i)
        plot_calendar_heatline(pair, df, ax=ax)
    plt.tight_layout()
    plt.show()
else:
    print('No 100% daily pairs to visualize.')

In [ ]:
# 2) Timeline/line plot (daily counts or presence) for a selected pair list (weekly and monthly similarly)
def plot_time_series(pair, df, freq='D', ax=None):
    a,b = pair
    mask = (df['AccountNumber'].astype(str) == str(a)) & (df['CorrespondentAccountNumber'].astype(str) == str(b))
    s = df[mask].copy()
    if s.empty:
        return None
    s = s.set_index('ValueDate').sort_index()
    if freq=='D':
        ts = s['AccountNumber'].resample('D').count()
    elif freq=='W':
        ts = s['AccountNumber'].resample('W').count()
    elif freq=='M':
        ts = s['AccountNumber'].resample('M').count()
    else:
        raise ValueError('freq must be D/W/M')
    if ax is None:
        fig, ax = plt.subplots(figsize=(12,3))
    ax.plot(ts.index, ts.values)
    ax.set_title(f'Transfer counts over time for {a} -> {b} (freq={freq})')
    ax.set_ylabel('count')
    return ax

# Plot for top N weekly pairs (if exist)
if weekly_pairs:
    n = len(weekly_pairs)
    fig = plt.figure(figsize=(12,3*n))
    for i, pair in enumerate(weekly_pairs, start=1):
        ax = fig.add_subplot(n,1,i)
        plot_time_series(pair, df, freq='W', ax=ax)
    plt.tight_layout()
    plt.show()
else:
    print('No 100% weekly pairs to visualize.')

In [ ]:
# 3) Heatmap per pair vs day (binary presence)
# Build a matrix for up to VISUAL_CAP pairs (rows) and all dates (cols)
def build_presence_matrix(pairs, df):
    dates = pd.date_range(df['ValueDate'].min().date(), df['ValueDate'].max().date(), freq='D')
    mat = []
    labels = []
    for a,b in pairs:
        mask = (df['AccountNumber'].astype(str)==str(a)) & (df['CorrespondentAccountNumber'].astype(str)==str(b))
        s = df[mask].groupby(df['ValueDate'].dt.date).size()
        row = [1 if d.date() in s.index else 0 for d in dates]
        mat.append(row)
        labels.append(f'{a}->{b}')
    return pd.DataFrame(mat, index=labels, columns=dates)

pairs_for_heatmap = list(set(daily_pairs + weekly_pairs + monthly_pairs))[:VISUAL_CAP]
if pairs_for_heatmap:
    presence_df = build_presence_matrix(pairs_for_heatmap, df)
    plt.figure(figsize=(14, max(3, 0.25*len(presence_df))))
    plt.imshow(presence_df.values, aspect='auto', interpolation='nearest')
    plt.yticks(range(len(presence_df.index)), presence_df.index)
    plt.xticks(rotation=45)
    plt.title('Presence heatmap (rows: pairs, cols: days)')
    plt.xlabel('Date')
    plt.colorbar()
    plt.tight_layout()
    plt.show()
else:
    print('No pairs available for heatmap.')

In [ ]:
# 4) Network graph for recurring pairs (combine all 100% pairs)
nodes = set()
edges = []
for dfp in [daily_100, weekly_100, monthly_100]:
    for _, row in dfp.iterrows():
        a = str(row['AccountNumber'])
        b = str(row['CorrespondentAccountNumber'])
        nodes.add(a); nodes.add(b)
        edges.append((a,b))

if edges and nx_available:
    G = nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    plt.figure(figsize=(12,8))
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, arrows=True, node_size=500, font_size=8)
    plt.title('Network of 100% recurring pairs (combined)')
    plt.show()
elif edges:
    print('networkx not available in the environment; cannot draw network graph.')
else:
    print('No recurring edges to draw in network graph.')

In [ ]:
# 5) Bar chart: top recurring pairs by pct_weekly (or pct_daily if weekly not present)
import numpy as np
# compute a 'score' for ordering: mean of pct_daily, pct_weekly, pct_monthly
final_df['score'] = final_df[['pct_daily','pct_weekly','pct_monthly']].mean(axis=1)
topN = final_df[final_df['pct_daily']==100].copy().sort_values('score', ascending=False).head(VISUAL_CAP)
if topN.empty:
    topN = final_df[final_df['pct_weekly']==100].copy().sort_values('score', ascending=False).head(VISUAL_CAP)
if topN.empty:
    topN = final_df[final_df['pct_monthly']==100].copy().sort_values('score', ascending=False).head(VISUAL_CAP)

if not topN.empty:
    labels = topN['AccountNumber'].astype(str) + '->' + topN['CorrespondentAccountNumber'].astype(str)
    vals = topN['score']
    plt.figure(figsize=(10, max(4, 0.4*len(labels))))
    plt.barh(range(len(vals)), vals)
    plt.yticks(range(len(vals)), labels)
    plt.xlabel('recurrence score (mean pct daily/weekly/monthly)')
    plt.title('Top recurring pairs (100% in at least one bucket prioritized)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print('No top recurring pairs found for bar chart.')

In [ ]:
# 6) Amount-based recurrence/stability check
# If 'Amount' column exists, compute cv (std/mean) per pair and show low-variance recurring pairs
if 'Amount' in df.columns:
    amt_stats = df.groupby(['AccountNumber','CorrespondentAccountNumber'])['Amount'].agg(['count','mean','std']).reset_index()
    amt_stats['cv'] = amt_stats['std'] / amt_stats['mean'].replace({0: pd.NA})
    # Merge with final_df to keep only 100% recurring pairs
    merged = final_df.merge(amt_stats, on=['AccountNumber','CorrespondentAccountNumber'], how='left')
    stable = merged[merged['cv'] <= 0.1].sort_values('cv').head(50)  # CV <= 0.1
    print('Pairs with low CV (<=0.1):', len(stable))
    if not stable.empty:
        from IPython.display import display
        display(stable[['AccountNumber','CorrespondentAccountNumber','count','mean','std','cv','pct_daily','pct_weekly','pct_monthly']])
else:
    print('No Amount column found; skipping amount-based stability check.')

### Final notes
- The notebook filters only pairs with **100% recurrence** per your request. Visualizations are capped to avoid generating thousands of plots — you can change `VISUAL_CAP` at the top of the visualization section.  
- Outputs saved: `/mnt/data/recurrence_results.xlsx` with sheets for all pairs and 100%-recurrence subsets.  
- To re-run or adapt thresholds, modify the filters (e.g., `pct_weekly >= 80`) in the "Filter 100% recurrence" cell.

You can download the notebook file and run it on your environment or in Colab (upload the transactions file to the same directory or to Colab's file system).